## make sure you have all libraries installed. 
use conda environment: /scratch/da2734/twitter/worldbank_twitter_environment.yml

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# loading filtered samples

In [1]:
# filtered contains 0.8G of data!!
import time
start_time = time.time()
import pyarrow.parquet as pq
from glob import glob
import os
country_code = 'US'
month = '2012-1'
path_to_data = '/scratch/spf248/twitter/data/classification/US/filtered/'
tweets_filtered=pq.ParquetDataset(glob(os.path.join(path_to_data,                                           
#                                            country_code,
#                                            month,
                                           '*.parquet'))).read().to_pandas()
tweets_filtered['tweet_id'] = tweets_filtered['tweet_id'].astype(int)
print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds')
print(tweets_filtered.shape)

time taken to load filtered sample: 386.1313638687134 seconds
(92121093, 11)


In [2]:
start_time = time.time()
tweets_filtered = tweets_filtered[['tweet_id', 'text']]
print('drop columns:', str(time.time() - start_time), 'seconds')

drop columns: 5.684360504150391 seconds


In [3]:
tweets_filtered.head()

,tweet_id,text
0,276933934124765184,Damn i have to much homework
1,277143632232992768,Does a bedazzler work on leather? Serious ques...
2,277150490276540416,RT @porcelain10: Washington Post D Milbank bas...
3,277151157208637440,Finally off work
4,277175194454462465,Irrational: No BioShock PS Vita until Sony and...


# loading random samples

In [1]:
# random contains 0.8G of data!!
import time
start_time = time.time()
import pyarrow.parquet as pq
from glob import glob
import os
country_code = 'US'
month = '2012-1'
path_to_data = '/scratch/spf248/twitter/data/classification/US/random/'
tweets_random=pq.ParquetDataset(glob(os.path.join(path_to_data,                                           
#                                            country_code,
#                                            month,
                                           '*.parquet'))).read().to_pandas()
tweets_random['tweet_id'] = tweets_random['tweet_id'].astype(int)
print('time taken to load random sample:', str(time.time() - start_time), 'seconds')
print(tweets_random.shape)

time taken to load random sample: 194.6565079689026 seconds
(92114009, 11)


In [2]:
start_time = time.time()
tweets_random = tweets_random[['tweet_id', 'text']]
print('drop columns:', str(time.time() - start_time), 'seconds')

drop columns: 5.201020002365112 seconds


In [6]:
tweets_random.head()

,tweet_id,text
0,367881326273105920,@shoebydoo32 I only left to go back home for t...
1,367881326281519105,oh my god bina id idnt read that u played GTA ...
2,367882185916702722,I Have To Make What I Think Is The Best Decisi...
3,367883553121394690,@elizrod_ that's from hard work 😉
4,367883867719348224,RT @MileenaSucks: Can I just lay out in the gr...


In [4]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_filtered = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'filtered*.csv')], ignore_index = True)
    print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds', model_output_filtered.shape)
    
    start_time = time.time()
    model_output_filtered = model_output_filtered[['tweet_id', 'second']]
    model_output_filtered.columns = ['tweet_id', column]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    if column == 'is_unemployed':
        start_time = time.time()
        merged_filtered = pd.merge(model_output_filtered, tweets_filtered, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
    else:
        start_time = time.time()
        merged_filtered = pd.merge(model_output_filtered, merged_filtered, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
            
    print(merged_filtered.tail(10))
            
#     break
    
    
start_time = time.time()
merged_filtered = merged_filtered.drop_duplicates(['text'])
print('drop dup:', str(time.time() - start_time), 'seconds', merged_filtered.shape)

start_time = time.time()
pickle.dump( merged_filtered, open( "./ONNXturk100M/ALL_ONNX_BERT_ST_merged_filtered_100m_jun24.pkl", "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')
        




 is_unemployed
/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/is_unemployed/
time taken to load filtered sample: 80.48429036140442 seconds (75999592, 3)
drop columns: 0.5115442276000977 seconds


NameError: name 'tweets_filtered' is not defined

In [8]:
del merged_filtered

# merging filtered tweets and taking top 10%

In [6]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


boundary_list = [i/20.0 for i in range(11)]
boundary_list.reverse()
print('boundary_list', boundary_list)

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    if column != 'job_offer': continue
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_filtered = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'filtered*.csv')], ignore_index = True)
    print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds', model_output_filtered.shape)
    
    start_time = time.time()
    model_output_filtered = model_output_filtered[['tweet_id', 'second']]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    start_time = time.time()
    merged_filtered = pd.merge(model_output_filtered, tweets_filtered, how='inner', on = 'tweet_id')
    print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)

#     print(merged_filtered.head())

    start_time = time.time()
    merged_filtered = merged_filtered.drop_duplicates(['text'])
    print('drop dup:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
    
    start_time = time.time()
    merged_filtered = merged_filtered.sort_values(by=['second'], ascending=False)
    print('sort:', str(time.time() - start_time), 'seconds', merged_filtered.shape)    
    
    start_time = time.time()
    merged_filtered_temp = merged_filtered[:30000000]
    print('top 10M truncate:', str(time.time() - start_time), 'seconds', merged_filtered.shape)   
    
    print(merged_filtered.tail(50))
    
    start_time = time.time()
    pickle.dump( merged_filtered_temp, open( "./ONNXturk100M/{}_ONNX_BERT_ST_merged_filtered_100m_jun22.pkl".format(column), "wb" ) )
    print('time taken to dump:', str(time.time() - start_time), 'seconds')
        
#     break


boundary_list [0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0.0]


 is_unemployed


 lost_job_1mo


 job_search


 is_hired_1mo


 job_offer
/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/job_offer/
time taken to load filtered sample: 29.686917066574097 seconds (75198670, 3)
drop columns: 0.5181810855865479 seconds
time taken to merge: 93.91530275344849 seconds (75198670, 3)
drop dup: 116.30996441841125 seconds (65461336, 3)
sort: 46.12218117713928 seconds (65461336, 3)
top 10M truncate: 0.0006086826324462891 seconds (65461336, 3)
                    tweet_id    second  \
52995136  425300563577044992  0.001509   
36788608  797919678882402305  0.001508   
18997625  888857163732590592  0.001508   
25206348  800899324926427136  0.001508   
21249400  690004041087213571  0.001508   
48565908  754686475032297474  0.001508   
30178463  371150526143950848  0.001508   
27092336  645982618584924160  0.001508   
59052375  577320419453243392  0.001508

KeyboardInterrupt: 

In [24]:
print(merged_filtered[1300000:20000000].head(30)['text'])
print(merged_filtered[1300000:20000000].tail(30)['text'])
start_time = time.time()
pickle.dump( merged_filtered[1300000:20000000], open( "./ONNXturk100M/job_offer_select_ONNX_BERT_ST_merged_filtered_100m_jun22.pkl".format(column), "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')

22519252    #nurse #jobs Charge Entry Specialist PRN, Midw...
67533711    ★ JOB ★ #hiring #job # Orlando - MED SURG Nurs...
61238633    #Sales #Job in #Murfreesboro, TN: Event Specia...
46007175    #Job Administrative Assistant II / Receptionis...
47273325    #Job #Spokane Lead General Engineer: Spokane V...
21470115    #CareerArc #Retail #Job alert: Prepared Food C...
57759454    #internship #Job alert: Accountant- Entry Leve...
13895362    We're #hiring! Click to apply: Shift Superviso...
49103383    Can you recommend anyone for this #job? Physic...
38065965    Social Media &amp; Community Management Intern...
36577254    #Gadsden #Jobs Shift Leader: Boaz Description*...
55358699    24 Seven #Marketing #Job: Brand (Product) Mana...
15203445    #Job Alert: #Hawaii #Jobs GENERAL OFFICE CLERK...
3743242     JobsinBoston_ #Boston #Job Data Entry Coordina...
64953159    We're #hiring! Click to apply: CDL-A Team Comp...
22480598    Nintendo #Marketing #Job: Assistant Manager, R...
38957127

In [ ]:
# merged_filtered

# merging random tweets

In [7]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_random = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'random*.csv')], ignore_index = True)
    print('time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random.shape)
    
    start_time = time.time()
    model_output_random = model_output_random[['tweet_id', 'second']]
    model_output_random.columns = ['tweet_id', column]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    if column == 'is_unemployed':
        start_time = time.time()
        merged_random = pd.merge(model_output_random, tweets_random, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)
    else:
        start_time = time.time()
        merged_random = pd.merge(model_output_random, merged_random, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)
            
    print(merged_random.tail(10))
            
#     break
    
    
start_time = time.time()
merged_random = merged_random.drop_duplicates(['text'])
print('drop dup:', str(time.time() - start_time), 'seconds', merged_random.shape)

start_time = time.time()
pickle.dump( merged_random, open( "./ONNXturk100M/ALL_ONNX_BERT_ST_merged_random_100m_jun24.pkl", "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')
        




 is_unemployed
/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/is_unemployed/
time taken to load random sample: 110.52857494354248 seconds (91060413, 3)
drop columns: 0.5876379013061523 seconds
time taken to merge: 89.58239603042603 seconds (91060413, 3)
                    tweet_id  is_unemployed  \
91060403  480800302723903489       0.005023   
91060404  480800424732413952       0.009104   
91060405  480800554541912064       0.010539   
91060406  480801041223401472       0.004538   
91060407  480801104012120064       0.003330   
91060408  480801754334756866       0.005141   
91060409  480801993267503104       0.006567   
91060410  480803981493420033       0.038735   
91060411  480805851922325505       0.004640   
91060412  480805906477613056       0.006376   

                                                       text  
91060403  RT @alexmorgan13: 2 more hours!! Me and @melea...  
91060404           a fuck nigga, thats the shit i dont like  
910

drop dup: 129.14112615585327 seconds (82905552, 7)
time taken to dump: 58.25591516494751 seconds
